$$
\begin{align}
V: & \text{set of vertices}\\
E: & \text{set of edges}\\
\Pi: & \text{set of partitions}\\
K: & \text{number of partitions } |\Pi|\\
\rho_{i}:  & \text{representative vertex $i$ of the partition}\\
\epsilon_{ij}:  & \text{edge between the vertices $i$ and $j$}\\
\omega_{ij}:  & \text{wieght of the edge between the vertices $i$ and $j$}\\
n(i): & \text{neighborhoods of the edge $i$}\\
\varphi: & \text{penalty for not create a partition}
\end{align}
$$


$$
\underset{\rho, \epsilon, s}\min \left\{z := \sum_{(i, j) \in E}\omega_{ij}\epsilon_{ij} + \varphi\big(K - s^- - s^+\big) \right\}, \text{ where } \varphi := \sum_{(i, j) \in E}\omega_{ij}
$$

$$
\begin{align}
\sum_{\pi \in \Pi} \rho_{i\pi} \leq 1 & & \forall i \in V\\
\sum_{i \in V} \rho_{i\pi} \leq 1 & & \forall \pi \in \Pi\\
\alpha\rho_{\alpha\pi} - \sum_{\pi \in \Pi}\left[\alpha\rho_{\beta\pi} + \beta(1 - \rho_{\beta\pi})\right] \leq \alpha\sum_{\gamma \in V}\rho_{\gamma\pi} \rho_{i\pi} & & \forall (\alpha, \beta, \pi) \in V \times V \times \Pi\\
\sum_{\pi \in \Pi}\sum_{i \in V} \rho_{i\pi} + s^- - s^+ = K & &\\
s^- + s^+ \leq K\\
\epsilon_{ij} + \sum_{k \in n(i)}\epsilon_{jk} \geq \rho_{i\pi}\big(|n(i)| - 1\big) & & \forall (i, j) \in E \times \Pi\\
\rho_{i\pi} \in \{0, 1\} & & \forall (i, \pi) \in V \times \Pi\\
\epsilon_{ij} \in \{0, 1\} & & \forall (i, j) \in E\\
s^-, s^+ \in \mathbb{R}^+
\end{align}
$$

$$
\alpha\rho_{\alpha\pi} \leq \sum_{\pi \in \Pi}\left(\beta(1 - \rho_{\beta\pi}) + \alpha\rho_{\beta\pi}\right)
$$


$$
\alpha\rho_{\alpha\pi} - \sum_{\pi \in \Pi}\left[\alpha\rho_{\beta\pi} + \beta(1 - \rho_{\beta\pi})\right] \leq \alpha\sum_{\gamma \in V}\rho_{\gamma\pi}
$$

In [ ]:
0 0: ok 
0 1: ok 
1 0: ok 
1 1: ok

In [ ]:
#A terceira restrição precisa de correções
#Algo que englobe o representante de menor indice da outra partição
#Se a partição existe
#Se o índice selecionado é o menor possível dado que os menores que ele 
#  já estão sendo utilizados em outras partições

In [2]:
#Restrições
# Um vértice pode representar no máximo uma partição
# Uma partição pode ter no máximo um representante
# Uma partição deve ter como representante o vértice de menor índice
# Desvio absoluto do número de partições em relação ao valor K
# Contendo o valor máximo das variáveis de folga
# Se um vértice é ligado à um representante então este vértice deve ser vizinho de todos
#   os vizinhos do representante. Por outro  lado, se ele não estiver conectado
#   à um representante então ele pode ser vizinho de qualquer um.